In [1]:
try:
    import folium
    from pyproj import Proj, transform
    import pandas as pd
except:
    !pip install folium
    !pip install pyproj

In [2]:
inProj = Proj(init='epsg:23031')

In [3]:
outProj = Proj(init='epsg:4326')

In [4]:
headers = [i.strip() for i in open("../Well_Tops").readlines()[5:33]]

In [5]:
df = pd.read_csv("../Well_Tops", delim_whitespace=True, skiprows =34, names=headers)

In [6]:
df["easting"], df["northing"] = transform(inProj, outProj, df.X.tolist(), df.Y.tolist())

In [7]:
wells = df.sort_values("Z", ascending=False).groupby("Well").first()

In [8]:
fmap = folium.Map(location=[wells.northing.iloc[0],
                            wells.easting.iloc[0]],
                  zoom_start=6,
                  tiles="cartodbpositron",width="100%", height="50%")

In [9]:
for x, y, name in zip(wells.easting, wells.northing, wells.index):    
    fg = folium.FeatureGroup(name=name)
    fg.add_child(folium.CircleMarker([y, x], tooltip=name,fill_color="red", color="black", radius=5))
    fmap.add_child(fg)
folium.LayerControl().add_to(fmap)
fmap